In [1]:
import os, sys
import splink.comparison_library as cl
from splink.exploratory import completeness_chart
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets
from splink_tools import *
from pprint import pprint 
import pandas as pd
import numpy as np
import json
from pprint import pprint



In [2]:
def move_working_dir_to_repo_root(repo_name="orgsync"):
    """
    Move the current working directory to the root of the repository.
    """
    current_dir = os.getcwd()
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")

USE_TEST_DATA = False
# base paths 
test_dir = os.path.join("data", "test", "processed")
data_dir = os.path.join("data", "raw", "processed")

# load data as json
if not USE_TEST_DATA:
    with open(os.path.join(data_dir, "gtr_data.json"), "r") as f:
        gtr_json = json.load(f)
    with open(os.path.join(data_dir, "cordis_data.json"), "r") as f:
        cordis_json = json.load(f)

else:
    with open(os.path.join(test_dir, "gtr_data.json"), "r") as f:
        gtr_json = json.load(f)

    with open(os.path.join(test_dir, "cordis_data.json"), "r") as f:
        cordis_json = json.load(f)

pprint(gtr_json[0])
pprint(cordis_json[0])

Current working directory:  c:\Users\dec2g\GitHub\OrgSync
{'address.postCode': 'NE1 8QH',
 'address.region': 'North East',
 'address.type': 'MAIN_ADDRESS',
 'id': '5331B126-3AB4-4412-B56D-00E8F2796556',
 'link.EMPLOYEE': ['0CB4A538-AC14-4394-A10F-C9F955033EF3'],
 'name': 'NEWCASTLE CITY COUNCIL'}
{'city': 'MOOR ROW',
 'geolocation': '',
 'name': 'NUCLEAR DECOMMISSIONING AUTHORITY - NDA',
 'nutsCode': '',
 'organisationID': 999565019,
 'postCode': 'CA24 3HU',
 'rcn': 1906596,
 'shortName': 'NDA',
 'street': 'Westlakes Science PArk - Herdus House'}


In [3]:
# create df from json with specifications indicating the ordering of columns by name, removal of columns, and mappings for changing the name of columns  
# gtr

gtr_df = pd.DataFrame(gtr_json)
gtr_df.head()

map_names_gtr = {
    "id": "unique_id",
    "address.postCode": "postcode"

}
drop_cols_gtr = ["link.EMPLOYEE","address.type","address.region"]
gtr_order = ["unique_id", "name"]


gtr_df = drop_cols(gtr_df, drop_cols_gtr)
gtr_df = change_col_names(gtr_df, map_names_gtr)
gtr_df = reorder_cols(gtr_df, gtr_order)
gtr_df.head(50)


,unique_id,name,postcode
0,5331B126-3AB4-4412-B56D-00E8F2796556,NEWCASTLE CITY COUNCIL,NE1 8QH
1,53331120-0290-49FA-A513-0286A214AF7A,VALERANN UK LIMITED,MK14 6GD
2,77874202-2018-4677-8CFF-0868CD838659,Baltic Sea Cultural Centre in Gdansk,None
3,77908BF8-1B2D-4D26-9119-155100E8B9C5,Mindray,PE29 6FN
4,7794C645-9CC7-4913-A8DC-103AE0EFDD4B,Democracy International,20814
5,77A41173-DC2B-4D30-A5E2-0B19BE4B8FD5,Wimbledon College of Art,None
6,53474E16-DA35-4007-9F07-1B6C0F32BBF6,LICX UK LIMITED,HP7 0UT
7,5370084A-BDBF-4A7B-8E02-09B7DE95D01B,PANITEK POWER LIMITED,EC1V 2NX
8,53700B68-0910-469D-A0CB-18A3316DF3F2,Agency for Healthcare Research and Quality,Unspecified
9,537D521C-3D73-458B-A054-196982ADD766,CONTROL DEVELOPMENTS (UK) LTD,OX29 4PU


In [4]:
for column_name in gtr_df.columns:
    print(column_name, check_column_datatypes(d=gtr_df, column=column_name))


unique_id unique_id
<class 'str'>    69067
Name: count, dtype: int64
name name
<class 'str'>    69067
Name: count, dtype: int64
postcode postcode
<class 'str'>         51570
<class 'NoneType'>    17497
Name: count, dtype: int64


In [5]:
cordis_df = pd.DataFrame(cordis_json)
cordis_df.head()

map_names_cordis = {
    # "organisationID": "unique_id",
    "rcn": "unique_id",
    "postCode": "postcode", 
}

drop_cols_cordis = ["geolocation", "nutsCode", "shortName", "street", "city", "organisationID"]

cordis_order = ["unique_id", "name", "postcode"]

cordis_df = drop_cols(cordis_df, drop_cols_cordis)
cordis_df = change_col_names(cordis_df, map_names_cordis)
cordis_df = reorder_cols(cordis_df, cordis_order)
cordis_df.head(50)

,unique_id,name,postcode
0,1906596,NUCLEAR DECOMMISSIONING AUTHORITY - NDA,CA24 3HU
1,2123954,SOUTH WEST TOURISM LIMITED,EX2 5WT
2,1915686,WELSH GOVERNMENT,CF10 3NQ
3,2128407,TERRASALUS LIMITED,LE15 9EL
4,2168344,Ol PHARMA PARTNERS LTD,SG4 7DP
5,1917645,FORSITE DIAGNOSTICS LTD,YO41 1LZ
6,1907389,ST GEORGE'S HOSPITAL MEDICAL SCHOOL,SW17 0RE
7,1909146,GENETIC ALLIANCE UK LTD,N1 3QP
8,2102071,FETAL MEDICINE FOUNDATION,W1G 6BG
9,2142093,Centre of the Cell,E1 2AB


In [6]:
for column_name in cordis_df.columns:
    print(column_name, check_column_datatypes(d=cordis_df, column=column_name))

unique_id unique_id
<class 'int'>    36508
Name: count, dtype: int64
name name
<class 'str'>    36508
Name: count, dtype: int64
postcode postcode
<class 'str'>    36506
<class 'int'>        2
Name: count, dtype: int64


In [7]:
# print all enties in cordis_df["postcode"] that are not strings
print(cordis_df[~cordis_df["postcode"].apply(lambda x: isinstance(x, str))])
# remove all entries in cordis_df["postcode"] that are not strings
cordis_df = cordis_df[cordis_df["postcode"].apply(lambda x: isinstance(x, str))]



      unique_id                                           name postcode
2106    2942505  FIA Foundation for the Automobile and Society    75005
3811    1924503                    Sembcorp Utilities (UK) Ltd     1985


In [8]:
cordis_df.head(50)

,unique_id,name,postcode
0,1906596,NUCLEAR DECOMMISSIONING AUTHORITY - NDA,CA24 3HU
1,2123954,SOUTH WEST TOURISM LIMITED,EX2 5WT
2,1915686,WELSH GOVERNMENT,CF10 3NQ
3,2128407,TERRASALUS LIMITED,LE15 9EL
4,2168344,Ol PHARMA PARTNERS LTD,SG4 7DP
5,1917645,FORSITE DIAGNOSTICS LTD,YO41 1LZ
6,1907389,ST GEORGE'S HOSPITAL MEDICAL SCHOOL,SW17 0RE
7,1909146,GENETIC ALLIANCE UK LTD,N1 3QP
8,2102071,FETAL MEDICINE FOUNDATION,W1G 6BG
9,2142093,Centre of the Cell,E1 2AB


In [9]:
check_empty_strings(cordis_df)


{'unique_id': np.False_, 'name': np.False_, 'postcode': np.True_}

In [10]:
cordis_df = blank_to_nan(cordis_df, column=None)
cordis_df.head(50)


,unique_id,name,postcode
0,1906596,NUCLEAR DECOMMISSIONING AUTHORITY - NDA,CA24 3HU
1,2123954,SOUTH WEST TOURISM LIMITED,EX2 5WT
2,1915686,WELSH GOVERNMENT,CF10 3NQ
3,2128407,TERRASALUS LIMITED,LE15 9EL
4,2168344,Ol PHARMA PARTNERS LTD,SG4 7DP
5,1917645,FORSITE DIAGNOSTICS LTD,YO41 1LZ
6,1907389,ST GEORGE'S HOSPITAL MEDICAL SCHOOL,SW17 0RE
7,1909146,GENETIC ALLIANCE UK LTD,N1 3QP
8,2102071,FETAL MEDICINE FOUNDATION,W1G 6BG
9,2142093,Centre of the Cell,E1 2AB


In [11]:
# count nans in each column using pandas
print(count_nans(cordis_df, column=None))
print(count_nans(gtr_df, column=None))


unique_id      0
name           0
postcode     182
dtype: int64
unique_id        0
name             0
postcode     17497
dtype: int64


In [12]:
cordis_df = clean_string_columns(cordis_df, columns=["name", "postcode"])
cordis_df.head(50)

,unique_id,name,postcode
0,1906596,nuclear decommissioning authority nda,ca24 3hu
1,2123954,south west tourism limited,ex2 5wt
2,1915686,welsh government,cf10 3nq
3,2128407,terrasalus limited,le15 9el
4,2168344,ol pharma partners ltd,sg4 7dp
5,1917645,forsite diagnostics ltd,yo41 1lz
6,1907389,st georges hospital medical school,sw17 0re
7,1909146,genetic alliance uk ltd,n1 3qp
8,2102071,fetal medicine foundation,w1g 6bg
9,2142093,centre of the cell,e1 2ab


In [13]:
gtr_df = clean_string_columns(gtr_df, columns=["name", "postcode"])
gtr_df.head(50)

,unique_id,name,postcode
0,5331B126-3AB4-4412-B56D-00E8F2796556,newcastle city council,ne1 8qh
1,53331120-0290-49FA-A513-0286A214AF7A,valerann uk limited,mk14 6gd
2,77874202-2018-4677-8CFF-0868CD838659,baltic sea cultural centre in gdansk,None
3,77908BF8-1B2D-4D26-9119-155100E8B9C5,mindray,pe29 6fn
4,7794C645-9CC7-4913-A8DC-103AE0EFDD4B,democracy international,20814
5,77A41173-DC2B-4D30-A5E2-0B19BE4B8FD5,wimbledon college of art,None
6,53474E16-DA35-4007-9F07-1B6C0F32BBF6,licx uk limited,hp7 0ut
7,5370084A-BDBF-4A7B-8E02-09B7DE95D01B,panitek power limited,ec1v 2nx
8,53700B68-0910-469D-A0CB-18A3316DF3F2,agency for healthcare research and quality,unspecified
9,537D521C-3D73-458B-A054-196982ADD766,control developments uk ltd,ox29 4pu


In [ ]:
def convert_none_type_to_nan(df):
    """
    Convert None type to np.nan in dataframe
    """
    return df.where(pd.notnull(df), None)


# Try just gtr_persons as a test, to see if we can match person_ids
# first combine the the lists from each row in PI_PER_ids and COI_PER_ids 



In [14]:
### Remove columns with very low linkage

for col in per_list_cols:
    gtr_persons = empty_list_to_nan_full(gtr_persons, col)

db_api = DuckDBAPI()
# Conver empty lists to null before running...#
completeness_chart(gtr_persons, db_api=db_api, table_names_for_chart=["gtr_persons"])




NameError: name 'per_list_cols' is not defined

In [ ]:
chart = completeness_chart(gtr_persons, db_api=db_api, table_names_for_chart=["gtr_persons"])
# pprint(chart.to_dict())

chart_dict = chart.to_dict()

# dataset = chart_dict.get("data")["name"]

for dataset, completeness  in chart_dict.get("datasets").items():
    # print(dataset)
    for col_info in completeness:
        if col_info["completeness"] == 0:
            # remove col from df
            gtr_persons.drop(col_info["column_name"], axis=1, inplace=True)
            print("\n")
    print("\n")

gtr_persons.head()

,person_id,PI_PER_ids,COI_PER_ids,EMPLOYED_ids,firstName,surname
0,0400AE80-674B-4068-A7D4-748DFA887CDE,"[1FF7C213-559F-4374-9932-069540778CFE, CF64FA1...","[A69BBDA8-5CE1-41A2-872C-122079954D8F, 86E4C8C...",[01F2924C-FFB4-481B-B8F0-31234D33F0FA],Tom,Tregenza
1,0400FA13-4E69-4BB4-AACC-B8E13051441D,[F8DE3E25-99F4-4C8F-A5DE-511C98674535],[6734F477-6EE2-4082-9765-ABB040C34AE1],[AFE1E6E9-738F-4616-9D07-0781DAC34046],Marianne,Yon
2,04F7BEB3-2C6E-4027-8B28-3CDB92DA0787,"[B5879D1D-8A34-44B1-9EA0-08FC0B4DDB1A, 4D01024...",[4163D0A1-31AE-40A7-89F5-1038BA805963],[6BE9EFE8-67BC-4376-9142-6AE1A9AA6498],Semira,Manaseki-Holland
3,04FC1BF4-0F97-4020-AEE6-0C4265BD96BE,[BDEB0211-F9D6-4F8B-80A1-5723764A22FB],"[8C11A853-2407-4BD8-8063-48B8A2EC010E, 5D6AEA7...",[595A5FEA-6A63-4445-BD0B-7CA0CC2EE7DF],Julia,Edgar
4,05031B56-1BEB-4FE5-9FE3-ECA990789279,"[80386E87-6008-484D-96D6-16751FBC1BCE, F9CF650...","[65793E04-59E6-4C98-97C7-118FD0D97EFE, 1B2B1A7...",[7FF630E0-3355-4076-9E47-528D4B0DBCB3],Mark,Leake


# Splink - Blocking Rules


In [ ]:
# gtr_orgs now contains PROJECT_ids, and EMPLOYEE_ids as lists of strings representing unique identifiers.
# Similarly, gtr_persons now contains EMPLOYED_ids, PI_PER_ids, and COI_PER_ids as lists of strings representing unique identifiers.
# We now want to create a new dataframe from gtr_orgs, which replacing PI_PER_ids




EMPLOYEE_ids
<class 'float'>    54720
<class 'str'>      14347
Name: count, dtype: int64
PROJECT_ids
<class 'str'>      67239
<class 'float'>     1828
Name: count, dtype: int64
EMPLOYED_ids
<class 'str'>    133565
Name: count, dtype: int64
PI_PER_ids
<class 'float'>    96842
<class 'str'>      36723
Name: count, dtype: int64
COI_PER_ids
<class 'float'>    80524
<class 'str'>      53041
Name: count, dtype: int64


In [ ]:
def return_rows_in_column_with_dtype(df, col, dtype):
    return df[df[col].apply(lambda x: isinstance(x, dtype))]

float_rows = return_rows_in_column_with_dtype(gtr_orgs, "EMPLOYEE_ids", float)
print(float_rows)

                                       name  \
2      Baltic Sea Cultural Centre in Gdansk   
3                                   Mindray   
4                   Democracy International   
5                  Wimbledon College of Art   
6                           LICX UK LIMITED   
...                                     ...   
69059                       POSTBIOTICS INC   
69061           SPEARHEAD MARKETING LIMITED   
69063      Liverpool John Moores University   
69064              University of Queensland   
69065                           Disyn Biotc   

                                             PROJECT_ids EMPLOYEE_ids  
2      ['6272E5E8-1321-48E3-B91B-63E358F4B4FD', '34D4...          NaN  
3               ['86D3E475-E582-4D08-8D13-99CEEFC43E17']          NaN  
4               ['B6328B17-69B0-4044-B156-0A26D338A5CE']          NaN  
5      ['7BFFAA57-86BD-423E-A20E-DE07A97F1366', '2616...          NaN  
6               ['A1026C17-835C-4C5E-848D-C592CCB035E7']          NaN  
...

In [ ]:
check_column_datatypes(gtr_orgs, "EMPLOYEE")

EMPLOYEE
<class 'float'>    54720
<class 'list'>     14347
Name: count, dtype: int64

### Analyse Missingness

It's important to understand the level of missingness in your data, because columns with higher levels of missingness are less useful for data linking.

In [ ]:
db_api = DuckDBAPI()
completeness_chart(gtr_orgs, db_api=db_api)
# Why is employee 100% complete? There should be a bunch of NaNs. 

alt.LayerChart(...)

# EDA


In [ ]:
from splink.exploratory import profile_columns

profile_columns(gtr_orgs, db_api=DuckDBAPI(), top_n=10, bottom_n=5)

alt.VConcatChart(...)